data source = "https://www.kaggle.com/datasets/mbkinaci/fruit-images-for-object-detection"

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

DATA LOADING

In [2]:
train_dir = "./fruits/train_zip/train/"
test_dir = "./fruits/test_zip/test/"

In [3]:
# fruit_dir = 
train_imgs = os.listdir("fruits/train_zip/train")
test_imgs = os.listdir("fruits/test_zip/test")

In [4]:
labels = []
for i in train_imgs:
    label = i.split('_')[0]
    if label in labels:
        pass
    else:
        labels.append(label)

In [5]:
labels

['orange', 'banana', 'mixed', 'apple']

In [6]:
image_size = 100

def get_data(imgs_path):
    data = []
    for img in os.listdir(imgs_path):
        im_s = img.split('.')[1]
        if (im_s == 'jpg') or (im_s == 'jpeg') or (im_s == 'png'):
            label = img.split('_')[0]
            label_index = labels.index(label)
            im_path = imgs_path+img
            img_arr = cv2.imread(im_path, cv2.IMREAD_COLOR)
            resize_img = cv2.resize(img_arr, (image_size, image_size))
            data.append([resize_img, label_index])
    return np.array(data)

In [7]:
training_data = get_data(train_dir)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
/tmp/ipykernel_6496/407720218.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [8]:
test_data = get_data(test_dir)

libpng warning: iCCP: known incorrect sRGB profile
/tmp/ipykernel_6496/407720218.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


**SPLIT DATA**

In [9]:
def get_x_and_y(data):
    X, y = [], []
    for feature, label in data:
        X.append(feature)
        y.append(label)
    return X, y

In [10]:
X_train, y_train = get_x_and_y(training_data)

In [11]:
X_test, y_test = get_x_and_y(test_data)

**NORMALIZE DATA**

Normalize X to form 0..1 for easier convergence by CNN.

In [12]:
def normalize(X):
    return np.array(X) / 255

In [13]:
X_train = normalize(X_train)
X_test = normalize(X_test)

**RESHAPE**

Reshape data to 3-D from 1-D as required by CNN

In [14]:
from sklearn.preprocessing import LabelBinarizer

In [15]:
def reshape_data(X, y):
    lb = LabelBinarizer()
    _x = X.reshape(-1, image_size, image_size, 3) # reshape to 3-d
    _y = lb.fit_transform(y)
    return _x, _y

In [16]:
X_train, y_train = reshape_data(X_train, y_train)

In [17]:
X_test, y_test = reshape_data(X_test, y_test)

## MODEL

**Training VGG-19 Model**

In [18]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet
import random
import tensorflow as tf

2023-01-13 13:37:22.079641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/charles/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-13 13:37:22.079676: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [29]:
INPUT_SHAPE = (image_size, image_size, 3)
EPOCHS = 20
BATCH_SIZE = 16

def fruits_model():
    model_trained = VGG19(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')
    for layer in model_trained.layers[:19]:
        layer.trainable = False

        model = Sequential(
            [
                model_trained,
                MaxPool2D((2,2), strides=2),
                Flatten(),
                Dense(4, activation='softmax')
            ]
        )
        model.compile(
            optimizer='adam',
            loss = 'categorical_crossentropy',
            metrics = ['accuracy']
        )
    return model

In [30]:
model = fruits_model()

2023-01-13 13:12:33.078412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/charles/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-13 13:12:33.078449: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-13 13:12:33.078482: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (charles-HP-EliteBook-Folio-9480m): /proc/driver/nvidia/version does not exist
2023-01-13 13:12:33.078909: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 1, 1, 512)         0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 2052      
Total params: 20,026,436
Trainable params: 4,721,668
Non-trainable params: 15,304,768
_________________________________________________________________


In [32]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 1, verbose=1,factor=0.3, min_lr=0.0005)

In [73]:
model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs = EPOCHS, validation_data = (X_test, y_test), callbacks = [learning_rate_reduction])

2023-01-13 02:24:21.752059: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
15/15 [==============================] - 67s 5s/step - loss: 1.2302 - accuracy: 0.5458 - val_loss: 0.5575 - val_accuracy: 0.7667
Epoch 2/20
15/15 [==============================] - 87s 6s/step - loss: 0.2996 - accuracy: 0.8708 - val_loss: 0.2289 - val_accuracy: 0.9167
Epoch 3/20
15/15 [==============================] - 72s 5s/step - loss: 0.1200 - accuracy: 0.9542 - val_loss: 0.2010 - val_accuracy: 0.9167

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005.
Epoch 4/20
15/15 [==============================] - 65s 4s/step - loss: 0.0583 - accuracy: 0.9792 - val_loss: 0.2168 - val_accuracy: 0.9167
Epoch 5/20
15/15 [==============================] - 68s 5s/step - loss: 0.0120 - accuracy: 0.9958 - val_loss: 0.1543 - val_accuracy: 0.9500
Epoch 6/20
15/15 [==============================] - 65s 4s/step - loss: 0.0086 - accuracy: 0.9958 - val_loss: 0.2056 - val_accuracy: 0.9167
Epoch 7/20
15/15 [==============================] - 64s 4s/step - loss: 0.0028 - accuracy: 1.0

In [74]:
model.save('../AI Models/fruits_classfication_20epochs.h5')

In [75]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 6s 3s/step - loss: 0.1858 - accuracy: 0.9667


[0.1857992261648178, 0.9666666388511658]

**PREDICTION**

In [81]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

In [83]:
y_pred[:20]

array([0, 0, 3, 0, 0, 0, 0, 1, 2, 1, 1, 1, 2, 1, 2, 1, 0, 3, 3, 0])

In [90]:
y = np.argmax(y_test, axis=-1)

In [91]:
y

array([0, 0, 3, 0, 0, 0, 0, 1, 3, 1, 1, 1, 2, 1, 2, 2, 0, 3, 3, 0, 3, 0,
       0, 2, 3, 0, 3, 3, 0, 3, 3, 1, 3, 3, 1, 0, 0, 3, 1, 0, 0, 3, 1, 1,
       3, 1, 1, 0, 1, 1, 1, 1, 3, 3, 0, 1, 2, 1, 3, 3])

**EVALUATION**

In [92]:
print(confusion_matrix(y, y_pred))

[[18  0  0  0]
 [ 0 18  0  0]
 [ 0  1  4  0]
 [ 0  0  1 18]]


In [93]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.95      1.00      0.97        18
           2       0.80      0.80      0.80         5
           3       1.00      0.95      0.97        19

    accuracy                           0.97        60
   macro avg       0.94      0.94      0.94        60
weighted avg       0.97      0.97      0.97        60



In [96]:
print(f"{accuracy_score(y, y_pred) * 100:0.2f} %")

96.67 %


## PREDICT ON UNSEEN DATA

In [19]:
# Load Unseen data
unseen_dir = "./unseen_data/"

In [20]:
data = get_data(unseen_dir)

/tmp/ipykernel_6496/407720218.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [23]:
X_u, y_u = get_x_and_y(data)

In [24]:
X_u = normalize(X_u)

In [25]:
X_u, y_u = reshape_data(X_u, y_u)

In [26]:
from tensorflow.keras.models import load_model

In [27]:
fruit_model = load_model("../AI Models/fruits_classfication_20epochs.h5")

2023-01-13 13:38:58.667167: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/charles/.local/lib/python3.8/site-packages/cv2/../../lib64:
2023-01-13 13:38:58.667266: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-13 13:38:58.667303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (charles-HP-EliteBook-Folio-9480m): /proc/driver/nvidia/version does not exist
2023-01-13 13:38:58.667702: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
fruit_model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 1, 1, 512)         0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 2052      
Total params: 20,026,436
Trainable params: 4,721,668
Non-trainable params: 15,304,768
_________________________________________________________________


In [30]:
y_pred_unseen = np.argmax(fruit_model.predict(X_u), axis=-1)

In [31]:
y_pred_unseen

array([1, 3, 0, 0, 3, 0, 1, 1, 0])

In [32]:
y_u

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0]])

In [33]:
y = np.argmax(y_u, axis=-1)

In [36]:
accuracy_score(y, y_pred_unseen)

0.6666666666666666

## CONCLUSION

Trained model has a final accuracy of 96% on test data which is very good. However, it performs poorly on Unseen data with accuracy of 66.66%. The reason for this perfomance is because it is unable to defferentiate between mixed fruits and multiple fruits in the same picture. Thus, if we have more than one banana in the same photo without any other fruit, the model classifies it has mixed instead of banana.

**What is solution to this issue?**

I think the best way to solve the issue is to have more additional data with images of plural fruits (of same species) and and retrain the model.


----------------------------------------------------------------------------------------**THE END**-------------------------------------------------------------------